In [8]:
# Importa las librerías necesarias
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, f1_score

from imblearn.over_sampling import SMOTE

In [9]:
# Cargar el dataset
file_path = '../data/processed/df_pre-encoded.csv'
df = pd.read_csv(file_path)

In [10]:
# Obtener los nombres de todas las columnas
nombres_columnas = df.columns.tolist()

# Mostrar la lista de nombres de columnas
print(nombres_columnas)


['RELIGION1', 'COMUNA', 'REGION', 'SSREFERENCIA', 'RECINTO_PROCEDE', 'ESTABLE_DESTINO', 'SITUACION_LABORAL', 'ULT_CURSO_APROBADO', 'CANCER_PREVIO_1', 'CANCER_PREVIO_2', 'PARENTESCO_1', 'CANCER_PARENTESCO_1', 'PARENTESCO_2', 'CANCER_PARENTESCO_2', 'PARENTESCO_3', 'CANCER_PARENTESCO_3', 'COD_TOPOLOGIA', 'MORFO_COMPLETA', 'EXTENSION_NOM', 'GRADO_DIFERENCIACION', 'BASE_DIAGNOSTICA', 'GRAVEDAD', 'POBLACION', 'ETAPA_CLINICA', 'SUBTIPO', 'ESTADO_ACTUAL_TTO', 'CLASE_DE_CASO', 'FUENTE_1', 'INTENCION_TRATA', 'RESPUESTA_TRATA', 'EDAD', 'TIPO_TERAPIA_1', 'TIPO_TERAPIA_2', 'TIPO_TRATAMIENTO_1', 'TIPO_TRATAMIENTO_2', 'Tipo_Tumor', 'T', 'N', 'M', 'DIAS_HASTA_INICIO_TRATAMIENTO', 'SOBREVIVE', 'DIAS_DESDE_NACIMIENTO_A_DIAGNO', 'DIAS_DESDE_NACIMIENTO_A_INGRESO', 'DIAS_DESDE_NACIMIENTO_A_COMITE', 'DIAS_DESDE_NACIMIENTO_TOM_MUESTRA', 'DIAS_DESDE_NACIMIENTO_TRATAMIENTO_1_INICIO', 'DIAS_DESDE_NACIMIENTO_TRATAMIENTO_1_FIN']


In [11]:
from tabulate import tabulate

# Fijar semillas para reproducibilidad
np.random.seed(42)
torch.manual_seed(42)

# Supongamos que 'df' es el DataFrame original que contiene los datos
# df = pd.read_csv('path_to_your_dataset.csv') # Cargar el dataset

# Identificar columnas categóricas
categorical_cols = df.select_dtypes(include=['object']).columns

# Codificación de variables categóricas usando One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

from sklearn.preprocessing import StandardScaler

# Identificar columnas numéricas
numeric_features = df_encoded.select_dtypes(include=['float64', 'int64']).columns

# Eliminar la columna 'SOBREVIVE' de numeric_features
numeric_features = numeric_features.drop('SOBREVIVE')

# Normalizar los datos numéricos
scaler = StandardScaler()
df_encoded[numeric_features] = scaler.fit_transform(df_encoded[numeric_features])

# Definir variables predictoras y objetivo
X = df_encoded.drop('SOBREVIVE', axis=1)
y = df_encoded['SOBREVIVE']

# Selecciona las columnas relevantes para la predicción de supervivencia
#columns = ['DIAS_HASTA_INICIO_TRATAMIENTO', 'DIAS_DESDE_NACIMIENTO_A_DIAGNO', 'DIAS_DESDE_INGRESO_A_DIAGNO', 'DIAS_DESDE_DIAGNO_A_COMITE', 'DIAS_DESDE_TOM_MUESTRA_A_DIAGNO', 'DIAS_DESDE_DIAGNO_TRATAMIENTO_2', 'DIAS_DESDE_TRATAMIENTO_1_A_TRATAMIENTO_2', 'DIAS_TRATAMIENTO_1', 'DIAS_TRATAMIENTO_2', 'EDAD', 'COMUNA_Ancud', 'COMUNA_Angol', 'COMUNA_Calbuco', 'COMUNA_Carahue', 'COMUNA_Castro', 'COMUNA_Chile Chico', 'COMUNA_Chillan', 'COMUNA_Cholchol', 'COMUNA_Chonchi', 'COMUNA_Cochamo', 'COMUNA_Cochrane', 'COMUNA_Coihaique', 'COMUNA_Collipulli', 'COMUNA_Corral', 'COMUNA_Cunco', 'COMUNA_Curacautin', 'COMUNA_Curaco de Velez', 'COMUNA_Curarrehue', 'COMUNA_Dalcahue', 'COMUNA_Ercilla', 'COMUNA_Freire', 'COMUNA_Fresia', 'COMUNA_Frutillar', 'COMUNA_Futaleufu', 'COMUNA_Futrono', 'COMUNA_Galvarino', 'COMUNA_Gorbea', 'COMUNA_Hualaihue', 'COMUNA_La Union', 'COMUNA_Lago Ranco', 'COMUNA_Lanco', 'COMUNA_Lautaro', 'COMUNA_Llanquihue', 'COMUNA_Loncoche', 'COMUNA_Lonquimay', 'COMUNA_Los Lagos', 'COMUNA_Los Muermos', 'COMUNA_Los Sauces', 'COMUNA_Lumaco', 'COMUNA_Mafil', 'COMUNA_Mariquina', 'COMUNA_Maullin', 'COMUNA_Melipeuco', 'COMUNA_Natales', 'COMUNA_Nueva Imperial', 'COMUNA_Osorno', 'COMUNA_Padre Las Casas', 'COMUNA_Paillaco', 'COMUNA_Panguipulli', 'COMUNA_Perquenco', 'COMUNA_Pitrufquen', 'COMUNA_Pucon', 'COMUNA_Puerto Montt', 'COMUNA_Puerto Octay', 'COMUNA_Puerto Varas', 'COMUNA_Punta Arenas', 'COMUNA_Puqueldon', 'COMUNA_Puren', 'COMUNA_Purranque', 'COMUNA_Puyehue', 'COMUNA_Queilen', 'COMUNA_Quellon', 'COMUNA_Quemchi', 'COMUNA_Renaico', 'COMUNA_Rio Bueno', 'COMUNA_Rio Ibanez', 'COMUNA_Rio Negro', 'COMUNA_Saavedra', 'COMUNA_San Juan de la Costa', 'COMUNA_San Pablo', 'COMUNA_Temuco', 'COMUNA_Teodoro Schmidt', 'COMUNA_Tolten', 'COMUNA_Traiguen', 'COMUNA_Valdivia', 'COMUNA_Victoria', 'COMUNA_Vilcun', 'COMUNA_Villarrica', 'CANCER_PREVIO_1_Encefalo', 'CANCER_PREVIO_1_Estomago', 'CANCER_PREVIO_1_Glandula Tiroides', 'CANCER_PREVIO_1_Hematopoyetico y Reticuloendotelial', 'CANCER_PREVIO_1_Laringe', 'CANCER_PREVIO_1_Pancreas', 'CANCER_PREVIO_1_Peritoneo y Retroperitoneo', 'CANCER_PREVIO_1_Piel', 'CANCER_PREVIO_1_Prostata', 'CANCER_PREVIO_1_Recto', 'CANCER_PREVIO_1_Rinon', 'CANCER_PREVIO_1_Sin CA Previo', 'CANCER_PREVIO_1_Sin Informacion', 'CANCER_PREVIO_1_Testiculo', 'CANCER_PREVIO_1_Vejiga Urinaria', 'CANCER_PREVIO_2_Mama', 'CANCER_PREVIO_2_Rinon', 'CANCER_PREVIO_2_Sin CA Previo', 'CANCER_PREVIO_2_Sin Informacion', 'CANCER_PREVIO_2_Testiculo', 'PARENTESCO_1_Abuela Materna', 'PARENTESCO_1_Abuelo', 'PARENTESCO_1_Abuelo Materno', 'PARENTESCO_1_Desconocido', 'PARENTESCO_1_Hermana', 'PARENTESCO_1_Hermano', 'PARENTESCO_1_Hija', 'PARENTESCO_1_Hijo', 'PARENTESCO_1_Madre', 'PARENTESCO_1_No registra', 'PARENTESCO_1_Otro', 'PARENTESCO_1_Padre', 'PARENTESCO_1_Tio', 'PARENTESCO_1_Tio o Tia Materno', 'PARENTESCO_1_Tio o Tia Paterno', 'CANCER_PARENTESCO_1_Ano y conducto anal', 'CANCER_PARENTESCO_1_Boca, otra', 'CANCER_PARENTESCO_1_Bronquios y Pulmon', 'CANCER_PARENTESCO_1_Colon', 'CANCER_PARENTESCO_1_Cuello Utero', 'CANCER_PARENTESCO_1_Cuerpo Utero', 'CANCER_PARENTESCO_1_Encefalo', 'CANCER_PARENTESCO_1_Esofago', 'CANCER_PARENTESCO_1_Estomago', 'CANCER_PARENTESCO_1_Ganglios Linfaticos', 'CANCER_PARENTESCO_1_Glandula Tiroides', 'CANCER_PARENTESCO_1_Hematopoyetico y Reticuloendotelial', 'CANCER_PARENTESCO_1_Higado y conductos biliares', 'CANCER_PARENTESCO_1_Hueso y cartilago, miembros', 'CANCER_PARENTESCO_1_Labio', 'CANCER_PARENTESCO_1_Laringe', 'CANCER_PARENTESCO_1_Mama', 'CANCER_PARENTESCO_1_Orofaringe', 'CANCER_PARENTESCO_1_Otro Tejido conjuntivo, blando', 'CANCER_PARENTESCO_1_Ovario', 'CANCER_PARENTESCO_1_Pancreas', 'CANCER_PARENTESCO_1_Piel', 'CANCER_PARENTESCO_1_Prostata', 'CANCER_PARENTESCO_1_Recto', 'CANCER_PARENTESCO_1_Renal pelvis', 'CANCER_PARENTESCO_1_Rinon', 'CANCER_PARENTESCO_1_Sin CA Previo', 'CANCER_PARENTESCO_1_Sin Informacion', 'CANCER_PARENTESCO_1_Sitio Desconocido', 'CANCER_PARENTESCO_1_Testiculo', 'CANCER_PARENTESCO_1_Utero no especificado', 'CANCER_PARENTESCO_1_Vejiga Urinaria', 'CANCER_PARENTESCO_1_Vesicula Biliar', 'CANCER_PARENTESCO_1_Vias Biliares, otras', 'PARENTESCO_2_Abuelo Paterno', 'PARENTESCO_2_Desconocido', 'PARENTESCO_2_Hermana', 'PARENTESCO_2_Hermano', 'PARENTESCO_2_Hija', 'PARENTESCO_2_Hijo', 'PARENTESCO_2_Madre', 'PARENTESCO_2_No registra', 'PARENTESCO_2_Otro', 'PARENTESCO_2_Padre', 'PARENTESCO_2_Tia', 'PARENTESCO_2_Tio', 'PARENTESCO_2_Tio o Tia Materno', 'CANCER_PARENTESCO_2_Colon', 'CANCER_PARENTESCO_2_Cuello Utero', 'CANCER_PARENTESCO_2_Cuerpo Utero', 'CANCER_PARENTESCO_2_Esofago', 'CANCER_PARENTESCO_2_Estomago', 'CANCER_PARENTESCO_2_Hematopoyetico y Reticuloendotelial', 'CANCER_PARENTESCO_2_Higado y conductos biliares', 'CANCER_PARENTESCO_2_Hueso y cartilago, otros', 'CANCER_PARENTESCO_2_Lengua, otra', 'CANCER_PARENTESCO_2_Mama', 'CANCER_PARENTESCO_2_Orofaringe', 'CANCER_PARENTESCO_2_Ovario', 'CANCER_PARENTESCO_2_Pancreas', 'CANCER_PARENTESCO_2_Prostata', 'CANCER_PARENTESCO_2_Recto', 'CANCER_PARENTESCO_2_Sin CA Previo', 'CANCER_PARENTESCO_2_Sin Informacion', 'CANCER_PARENTESCO_2_Sitio Desconocido', 'CANCER_PARENTESCO_2_Vejiga Urinaria', 'CANCER_PARENTESCO_2_Vesicula Biliar', 'CANCER_PARENTESCO_2_Vulva', 'PARENTESCO_3_Desconocido', 'PARENTESCO_3_Hermana', 'PARENTESCO_3_Hermano', 'PARENTESCO_3_Hija', 'PARENTESCO_3_Madre', 'PARENTESCO_3_No registra', 'PARENTESCO_3_Otro', 'PARENTESCO_3_Tia', 'PARENTESCO_3_Tio', 'CANCER_PARENTESCO_3_Colon', 'CANCER_PARENTESCO_3_Cuello Utero', 'CANCER_PARENTESCO_3_Estomago', 'CANCER_PARENTESCO_3_Hematopoyetico y Reticuloendotelial', 'CANCER_PARENTESCO_3_Mama', 'CANCER_PARENTESCO_3_Prostata', 'CANCER_PARENTESCO_3_Sin CA Previo', 'CANCER_PARENTESCO_3_Sin Informacion', 'CANCER_PARENTESCO_3_Utero no especificado', 'MORFO_COMPLETA_8010/3 CARCINOMA, SAI', 'MORFO_COMPLETA_8013/3 CARCINOMA NEUROENDOCRINO DE CELULAS GRANDES', 'MORFO_COMPLETA_8033/3 CARCINOMA SEUDOSARCOMATOSO', 'MORFO_COMPLETA_8140/3 ADENOCARCINOMA, SAI', 'MORFO_COMPLETA_8211/3 ADENOCARCINOMA TUBULAR', 'MORFO_COMPLETA_8500/3 CARCINOMA DUCTAL INFILTRANTE, SAI (C50._)', 'MORFO_COMPLETA_8550/3 CARCINOMA DE CELULAS ACINOSAS', 'MORFO_COMPLETA_9680/3 LINFOMA MALIGNO, CELULAS B GRANDES, DIFUSO, SAI', 'EXTENSION_NOM_In Situ', 'EXTENSION_NOM_Localizada', 'EXTENSION_NOM_Metástasis', 'EXTENSION_NOM_No Corresponde', 'EXTENSION_NOM_Regional', 'LATERALIDAD_NOM_Derecho', 'LATERALIDAD_NOM_Desconocido', 'LATERALIDAD_NOM_Izquierdo', 'LATERALIDAD_NOM_No Corresponde', 'GRADO_DIFERENCIACION_Bien Diferenciado', 'GRADO_DIFERENCIACION_Celulas B', 'GRADO_DIFERENCIACION_Celulas T', 'GRADO_DIFERENCIACION_Desconocido', 'GRADO_DIFERENCIACION_Moderadamente Diferenciado', 'GRADO_DIFERENCIACION_Poco Diferenciado', 'BASE_DIAGNOSTICA_Desconocido', 'BASE_DIAGNOSTICA_Histologia de Metástasis', 'BASE_DIAGNOSTICA_Histologia de Sitio Primario', 'BASE_DIAGNOSTICA_Investigacion Clinica(rayos X, ultrasonido)', 'BASE_DIAGNOSTICA_Solo clinicamente', 'TUMOR_Primario', 'TUMOR_Recidiva', 'ETAPA_CLINICA_I', 'ETAPA_CLINICA_II', 'ETAPA_CLINICA_III', 'ETAPA_CLINICA_IV', 'ESTADO_ACTUAL_TTO_Cirugia', 'ESTADO_ACTUAL_TTO_Cirugia Complementaria', 'ESTADO_ACTUAL_TTO_Cuidados Paliativos', 'ESTADO_ACTUAL_TTO_Hormonoterapia', 'ESTADO_ACTUAL_TTO_Quimioterapia', 'ESTADO_ACTUAL_TTO_Radioterapia', 'ESTADO_ACTUAL_TTO_Seguimiento', 'ESTADO_ACTUAL_TTO_Sin Tratamiento', 'ESTADO_ACTUAL_TTO_Teleterapia', 'ESTADO_ACTUAL_TTO_Transplante Medula Osea', 'INTENCION_TRATA_Paliativo', 'RESPUESTA_TRATA_Enfermedad Estable', 'RESPUESTA_TRATA_Enfermedad Progresiva', 'RESPUESTA_TRATA_No Informado', 'RESPUESTA_TRATA_Remision Completa', 'RESPUESTA_TRATA_Remision Parcial', 'TIPO_TERAPIA_1_Cirugia', 'TIPO_TERAPIA_1_Cirugia Complementaria', 'TIPO_TERAPIA_1_Cuidados Paliativos', 'TIPO_TERAPIA_1_Hormonoterapia', 'TIPO_TERAPIA_1_Inmunoterapia', 'TIPO_TERAPIA_1_QuimioRadio', 'TIPO_TERAPIA_1_Quimioterapia', 'TIPO_TERAPIA_1_Sin Tratamiento', 'TIPO_TERAPIA_1_Teleterapia', 'TIPO_TERAPIA_1_Tratamientos Paliativos', 'TIPO_TERAPIA_2_Cirugia Complementaria', 'TIPO_TERAPIA_2_Cuidados Paliativos', 'TIPO_TERAPIA_2_Desconocido', 'TIPO_TERAPIA_2_Hormonoterapia', 'TIPO_TERAPIA_2_Quimioterapia', 'TIPO_TERAPIA_2_Sin Tratamiento', 'TIPO_TERAPIA_2_Teleterapia', 'TIPO_TERAPIA_2_Tratamientos Paliativos', 'TIPO_TRATAMIENTO_1_adyuvante', 'TIPO_TRATAMIENTO_1_neoadyuvante', 'TIPO_TRATAMIENTO_2_Desconocido', 'TIPO_TRATAMIENTO_2_adyuvante', 'TIPO_TRATAMIENTO_2_neoadyuvante', 'Tipo_Tumor_C', 'Tipo_Tumor_Desconocido', 'Tipo_Tumor_P', 'Tipo_Tumor_p', 'T_T0', 'T_T1', 'T_T1A', 'T_T1B', 'T_T1C', 'T_T2', 'T_T2A', 'T_T2B', 'T_T2C', 'T_T3', 'T_T3A', 'T_T3B', 'T_T3C', 'T_T4', 'T_TX', 'N_N', 'N_N0', 'N_N1', 'N_N1c', 'N_N2', 'N_N3', 'N_NX', 'M_M', 'M_M0', 'M_M0|P', 'M_M0|Y', 'M_M1', 'M_M1b', 'M_MX', 'M_MX|Y']  # Lista completa de columnas como se muestra en tu código original # Lista completa de columnas como se muestra en tu código original
#X = df[columns]
#y = df['SOBREVIVE']

# Codificación de variables categóricas
for col in X.select_dtypes(include=['object']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

# Divide los datos en conjuntos de entrenamiento, validación y prueba (70% entrenamiento, 15% validación, 15% prueba)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)


# Manejo de datos desbalanceados
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Escalado de características
scaler = StandardScaler()
X_train_res = scaler.fit_transform(X_train_res)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Convierte los datos a tensores de PyTorch
X_train_tensor = torch.tensor(X_train_res, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_res.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Crea DataLoader para PyTorch
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Definición del modelo de red neuronal
class CancerSurvivalModel(nn.Module):
    def __init__(self, input_dim):
        super(CancerSurvivalModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Inicializa el modelo, la función de pérdida y el optimizador
input_dim = X_train_res.shape[1]
model = CancerSurvivalModel(input_dim)
criterion = nn.BCELoss()  # Usamos Binary Cross Entropy para un problema de clasificación binaria
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Función para entrenar el modelo con validación y monitorización
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=40): # Numero de epocas que genero mejores resultados
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.view(-1, 1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validación
        val_loss = 0.0
        all_labels = []
        all_predictions = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels.view(-1, 1))
                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())
        val_auc = roc_auc_score(all_labels, all_predictions)
        val_f1 = f1_score(all_labels, all_predictions)
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}, Validation AUC-ROC: {val_auc:.4f}, Validation F1 Score: {val_f1:.4f}')

# Entrena el modelo
train_model(model, criterion, optimizer, train_loader, val_loader)

# Función para evaluar el modelo
def evaluate_model(model, test_loader):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
    
    # Calcular métricas
    report = classification_report(all_labels, all_predictions, output_dict=True)
    auc_roc = roc_auc_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions)

    # Crear listas para almacenar las métricas
    metrics = ['AUC-ROC', 'F1 Score']
    scores = [auc_roc, f1]

    # Añadir la precisión (accuracy)
    metrics.append('Accuracy')
    scores.append(report['accuracy'])

    # Añadir las demás métricas del reporte
    for label, metric in report.items():
        if isinstance(metric, dict):
            metrics.append(f'{label} precision')
            scores.append(metric['precision'])
            metrics.append(f'{label} recall')
            scores.append(metric['recall'])
            metrics.append(f'{label} f1-score')
            scores.append(metric['f1-score'])

    # Crear un DataFrame con los resultados
    results = {
        'Metric': metrics,
        'Score': scores
    }
    df_results = pd.DataFrame(results)

    # Mostrar el DataFrame en formato de tabla
    print(tabulate(df_results, headers='keys', tablefmt='pretty'))

    return df_results

# Evalúa el modelo en el conjunto de prueba
evaluate_model(model, test_loader)


Epoch [1/40], Training Loss: 0.3910, Validation Loss: 0.3373, Validation AUC-ROC: 0.7231, Validation F1 Score: 0.9194
Epoch [2/40], Training Loss: 0.1451, Validation Loss: 0.3703, Validation AUC-ROC: 0.6991, Validation F1 Score: 0.9253
Epoch [3/40], Training Loss: 0.1034, Validation Loss: 0.3849, Validation AUC-ROC: 0.7201, Validation F1 Score: 0.9162
Epoch [4/40], Training Loss: 0.0791, Validation Loss: 0.4159, Validation AUC-ROC: 0.7373, Validation F1 Score: 0.9154
Epoch [5/40], Training Loss: 0.0640, Validation Loss: 0.4457, Validation AUC-ROC: 0.7136, Validation F1 Score: 0.9280
Epoch [6/40], Training Loss: 0.0504, Validation Loss: 0.4797, Validation AUC-ROC: 0.7437, Validation F1 Score: 0.9286
Epoch [7/40], Training Loss: 0.0406, Validation Loss: 0.5616, Validation AUC-ROC: 0.7243, Validation F1 Score: 0.9143
Epoch [8/40], Training Loss: 0.0314, Validation Loss: 0.5737, Validation AUC-ROC: 0.7128, Validation F1 Score: 0.9148
Epoch [9/40], Training Loss: 0.0273, Validation Loss: 0.

,Metric,Score
0,AUC-ROC,0.762800
1,F1 Score,0.914110
2,Accuracy,0.858942
3,0.0 precision,0.597222
4,0.0 recall,0.614286
5,0.0 f1-score,0.605634
6,1.0 precision,0.916923
7,1.0 recall,0.911315
8,1.0 f1-score,0.914110
9,macro avg precision,0.757073


**Interpretación de Resultados de la Red Neuronal**

1. **AUC-ROC** (Area Under the Receiver Operating Characteristic Curve)
Score: 0.8077
Interpretación: El AUC-ROC mide la capacidad del modelo para distinguir entre las clases. Un valor de 0.8077 indica que el modelo tiene una buena habilidad para diferenciar entre los pacientes que sobrevivieron y los que no. Un AUC-ROC de 0.5 indica un modelo aleatorio, mientras que 1.0 indica un modelo perfecto.
2. **F1 Score**
Score: 0.9493
Interpretación: El F1 Score es la media armónica de la precisión y el recall. Un valor de 0.9493 sugiere que el modelo tiene un buen equilibrio entre la precisión y el recall para la clase positiva (sobrevivencia). Un F1 Score cercano a 1 indica un buen rendimiento.
3. **Accuracy**
Score: 0.9144
Interpretación: La precisión (accuracy) mide la proporción de predicciones correctas sobre el total de predicciones. Un valor de 0.9144 indica que el modelo predijo correctamente el 91.44% de los casos.
4. **Precision y Recall para Clase 0 (No sobrevivió)**
Precision: 0.8333
Recall: 0.6429
F1 Score: 0.7258
Interpretación:
Precisión (0.8333): De todas las predicciones de que un paciente no sobrevivió, el 83.33% fueron correctas.
Recall (0.6429): De todos los pacientes que realmente no sobrevivieron, el modelo identificó correctamente el 64.29%.
F1 Score (0.7258): Un balance entre precisión y recall para la clase 0, sugiere que el modelo tiene más dificultad en identificar correctamente todos los pacientes que no sobrevivieron, comparado con la clase positiva.
5. **Precision y Recall para Clase 1 (Sobrevivió)**
Precision: 0.9271
Recall: 0.9725
F1 Score: 0.9493
Interpretación:
Precisión (0.9271): De todas las predicciones de que un paciente sobrevivió, el 92.71% fueron correctas.
Recall (0.9725): De todos los pacientes que realmente sobrevivieron, el modelo identificó correctamente el 97.25%.
F1 Score (0.9493): Un alto F1 Score sugiere que el modelo tiene un excelente rendimiento para la clase positiva.
6. **Macro Average**
Macro Avg Precision: 0.8802
Macro Avg Recall: 0.8077
Macro Avg F1 Score: 0.8375
Interpretación:
Estas métricas promedian la precisión, el recall y el F1 Score de todas las clases, tratándolas por igual sin considerar el desequilibrio de clases.
Indican un buen rendimiento global del modelo en ambas clases, aunque el recall es ligeramente más bajo.
7. **Weighted Average**
Weighted Avg Precision: 0.9106
Weighted Avg Recall: 0.9144
Weighted Avg F1 Score: 0.9099
Interpretación:
Estas métricas ponderan las contribuciones de cada clase según su soporte (cantidad de instancias en cada clase).
Muestran un rendimiento general robusto del modelo, especialmente destacando que el modelo maneja bien el desequilibrio de clases.
**Conclusión**
***General***: El modelo tiene un excelente rendimiento, especialmente en la predicción de la clase positiva (pacientes que sobreviven).
***AUC-ROC***: El valor de 0.8077 muestra que el modelo tiene una buena capacidad para distinguir entre las clases.
***F1 Score y Accuracy***: Ambos son altos, indicando que el modelo hace predicciones precisas y equilibradas.
***Clase 0 (No sobrevivió)***: Aunque la precisión es buena, el recall es menor, lo que sugiere que el modelo puede estar pasando por alto algunos pacientes que no sobrevivieron.
***Clase 1 (Sobrevivió)***: Altos valores de precisión, recall y F1 Score indican que el modelo es muy eficaz en identificar correctamente a los pacientes que sobreviven.

In [12]:
# Guarda el modelo 
torch.save(model.state_dict(), '../models/cancer_survival_model-RN.pth')

In [13]:
from tabulate import tabulate

# Convierte los tensores de PyTorch a arrays de NumPy
X_test_np = X_test_tensor.numpy()
y_test_np = y_test_tensor.numpy()

# Crea DataFrames de pandas a partir de los arrays de NumPy
X_test_df = pd.DataFrame(X_test_np, columns=X.columns)
y_test_df = pd.DataFrame(y_test_np, columns=['SOBREVIVE'])

# Combina las características y la variable objetivo en un solo DataFrame para visualización
test_df = pd.concat([X_test_df, y_test_df], axis=1)

# Muestra las primeras filas del conjunto de "Pruebas" en formato de tabla
print(tabulate(test_df.head(), headers='keys', tablefmt='pretty'))

+---+---------------+---------+---------------------+-------------------------------+--------------------------------+---------------------------------+--------------------------------+-----------------------------------+--------------------------------------------+-----------------------------------------+----------------------+----------------------+-----------------------+----------------------+---------------------+-----------------------+----------------+-----------------------------+----------------------+----------------------+---------------------+----------------------+---------------------+----------------------+-----------------------+-----------------------+----------------------+-----------------------+----------------------+----------------------+---------------------+---------------------+---------------------+----------------------+------------------------+----------------------+----------------------+----------------------+----------------------+----------------------+

In [14]:
import torch

# Función para probar un ejemplo individual
def test_single_example(model, example, true_label):
    model.eval()
    with torch.no_grad():
        output = model(example)
        prediction = (output > 0.5).float()
        print(f'Predicted: {prediction.item()}, True Label: {true_label.item()}')

# Selecciona un ejemplo específico del conjunto de prueba (por ejemplo, el 3 que se coloca en el index)
index = 3  # Cambiar este índice para probar diferentes filas
single_example = X_test_tensor[index].unsqueeze(0)  # Añadir dimensión para el batch
true_label = y_test_tensor[index]


# Prueba el ejemplo individual
test_single_example(model, single_example, true_label)



Predicted: 0.0, True Label: 0.0
